In [1]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [2]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [3]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [4]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [5]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [6]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:07,  1.12s/it]


In [7]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.185605,3.185605,0.000000,9.185605,-4.814395,-0.814395,0.0,7.185605,...,9.185605,-4.814395,-0.814395,0.0,7.185605,3.185605,0.000000,9.185605,-4.814395,-0.814395
1,2025-01-22,0.0,7.185605,3.185605,0.000000,9.185605,-4.814395,-0.814395,0.0,7.185605,...,9.185605,-4.814395,-0.814395,0.0,7.185605,3.185605,0.000000,9.185605,-4.814395,-0.814395
2,2025-01-23,0.0,0.884298,3.185605,-15.642504,1.456742,-4.814395,-0.814395,0.0,-6.936606,...,-8.135980,-4.814395,-0.814395,0.0,8.705202,3.185605,-15.642504,11.049465,-4.814395,-0.814395
3,2025-01-24,0.0,0.884298,3.185605,-15.642504,1.456742,-4.814395,-0.814395,0.0,-6.936606,...,-8.135980,-4.814395,-0.814395,0.0,8.705202,3.185605,-15.642504,11.049465,-4.814395,-0.814395
4,2025-01-25,0.0,0.884298,3.185605,-15.642504,1.456742,-4.814395,-0.814395,0.0,-6.936606,...,-8.135980,-4.814395,-0.814395,0.0,8.705202,3.185605,-15.642504,11.049465,-4.814395,-0.814395


In [8]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [9]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [10]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls['partisan'] = pres_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + pres_polls['partisan'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://napolitannews.org/jobapproval...",2026-01-28,2026-02-05,3000 RV,9.466520e-02,46.0,53.0,-7.0,-17.520617
1,"<a href=""https://www.politico.com/newsletters/...",2026-02-06,2026-02-09,1000 RV,8.290441e-02,43.0,57.0,-14.0,-16.563078
2,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-02-06,2026-02-09,1730 A,6.681322e-02,37.0,56.0,-19.0,-17.698609
3,"<a href=""https://pro.morningconsult.com/tracke...",2026-02-02,2026-02-08,45000 A,5.873142e-02,40.0,54.0,-14.0,-17.812347
4,"<a href=""https://pro.morningconsult.com/tracke...",2026-02-06,2026-02-09,2200 RV,5.019787e-02,44.0,53.0,-9.0,-14.195782
...,...,...,...,...,...,...,...,...,...
566,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,8.874829e-21,52.0,43.0,9.0,4.704717
567,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,6.858171e-21,49.0,36.0,13.0,8.426267
568,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,6.733157e-21,47.0,41.0,6.0,9.477614
569,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,6.692084e-21,51.9,45.3,6.6,-0.528651


In [11]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [12]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://pro.morningconsult.com/tracke...",2026-02-06,2026-02-09,Trade/tariffs,2200 RV,1.916259e-02,44.0,46.0,-2.0,-15.819763
1,"<a href=""https://johnzogbystrategies.com/the-p...",2026-02-04,2026-02-05,Trade/tariffs,1000 LV,1.529467e-02,40.0,55.0,-15.0,-23.118534
2,"<a href=""https://law.marquette.edu/assets/comm...",2026-01-21,2026-01-28,Trade/tariffs,1003 A,1.502721e-02,37.0,62.0,-25.0,-19.788997
3,"<a href=""https://harvardharrispoll.com/wp-cont...",2026-01-28,2026-01-29,Trade/tariffs,2000 RV,1.134370e-02,39.0,54.0,-15.0,-22.863962
4,"<a href=""https://static.foxnews.com/foxnews.co...",2026-01-23,2026-01-26,Trade/tariffs,1005 RV,9.461954e-03,37.0,63.0,-26.0,-23.711476
...,...,...,...,...,...,...,...,...,...,...
1135,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,1.006439e-14,47.0,37.0,10.0,7.330227
1136,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,5.825711e-15,48.0,37.0,11.0,8.330227
1137,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,3.377393e-15,45.0,41.0,4.0,1.330227
1138,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,1.979111e-15,48.0,35.0,13.0,10.330227


In [13]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [14]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['partisan'] = gb_polls['partisan'].replace({
    'DEM': ' (D)',
    'REP': ' (R)',
    'WFP': ' (WF)'
}).fillna('')
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + gb_polls['partisan'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-02-06,2026-02-09,1551 RV,1.248842e-01,44.0,38.0,-6.0,-5.961154
1,"<a href=""https://pro.morningconsult.com/tracke...",2026-02-02,2026-02-08,29303 RV,1.159398e-01,45.0,42.0,-3.0,-5.254085
2,"<a href=""https://johnzogbystrategies.com/democ...",2026-02-04,2026-02-05,1000 LV,9.494917e-02,47.0,42.0,-5.0,-5.687603
3,"<a href=""https://www.cygn.al/polls/national-po...",2026-01-27,2026-01-28,1004 LV,6.161468e-02,48.0,44.0,-4.0,-5.443947
4,"<a href=""https://law.marquette.edu/poll/2026/0...",2026-01-21,2026-01-28,577 LV,5.464383e-02,52.0,45.0,-7.0,-6.207430
...,...,...,...,...,...,...,...,...,...
291,"<a href=""https://www.cygn.al/voters-back-kingm...",2025-01-09,2025-01-12,1500 LV,1.379809e-20,44.6,46.8,2.2,1.154958
292,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,1.011490e-20,40.0,48.0,8.0,2.658289
293,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,5.177924e-21,45.0,48.0,3.0,1.074497
294,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,3.999403e-22,44.6,48.4,3.8,2.754958


In [15]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)